In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Using the patsy library
Developing interaction factors is a pretty common feature of many data science platforms and this can of course be done with Python as well.  The issue is that this requires a lot of `for` loops or recursive functions to develop all the interactions with the various variables. 

(from the [patsy documentation](https://patsy.readthedocs.io/en/latest/overview.html))
>`patsy` is a Python package for describing statistical models (especially linear models, or models that have a linear component) and building design matrices. It is closely inspired by and compatible with the formula mini-language used in R and S.
>
>For instance, if we have some variable y, and we want to regress it against some other variables x, a, b, and the interaction of a and b, then we simply write:
>
>```patsy.dmatrices("y ~ x + a + b + a:b", data)```
>
> and Patsy takes care of building appropriate matrices. Furthermore, it:
> 
> * Allows data transformations to be specified using arbitrary Python code: instead of `x`, we could have written `log(x)`, `(x > 0)`, or even `log(x) if x > 1e-5 else log(1e-5)`,
> * Provides a range of convenient options for coding categorical variables, including automatic detection and removal > of redundancies,
> * Knows how to apply ‘the same’ transformation used on original data to new data, even for tricky transformations like > centering or standardization (critical if you want to use your model to make predictions),
> * Has an incremental mode to handle data sets which are too large to fit into memory at one time,
> * Provides a language for symbolic, human-readable specification of linear constraint matrices,
> * Has a thorough test suite (>97% statement coverage) and solid underlying theory, allowing it to correctly handle > corner cases that even R gets wrong, and
> * Features a simple API for integration into statistical packages.
> 
> What Patsy won’t do is, well, statistics — it just lets you describe models in general terms. It doesn’t know or 
> care whether you ultimately want to do linear regression, time-series analysis, or fit a forest of decision trees,
>  and it certainly won’t do any of those things for you — it just gives a high-level language for describing which 
> factors you want your underlying model to take into account. It’s not suitable for implementing arbitrary non-linear 
> models from scratch; for that, you’ll be better off with something like Theano, SymPy, or just plain Python. 
> But if you’re using a statistical package that requires you to provide a raw model matrix, then you can use Patsy to painlessly construct that model matrix; 

In [21]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from patsy.highlevel import dmatrices

import sys
sys.path.append('..')
from src.data import load_excel, TRUE_VALUES, FALSE_VALUES
from src.metric import confusion_matrix,classificationSummary
from sklearn.neighbors import KNeighborsClassifier

pd.set_option('precision',4)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [10]:
# Import and clean-up the heart disease dataset
heart_df = load_excel('HeartDisease_Cleveland',
                    dtype={'FBS': bool, 'EXANG': bool}, true_values=TRUE_VALUES,
                    false_values=FALSE_VALUES, na_values=['?'])
heart_df.dropna(inplace=True)
heart_df['DIAG'] = (heart_df.NUM > 0)
heart_df.drop(columns=['NUM'], inplace=True)
# Set the MAX_NEIGHBORS
MAX_NEIGHBORS = 25


In [13]:
# This function will build the matrices (that is the predictors with interactions)
def setup_matricies(df):

    # We'll look for DIAG as the target and 
    # Define SEX, CP, FBS, RESTECG, EXANG, SLOPE and THAL as categorical variables
    # The other predictors are already seen as numeric
    y, X = dmatrices('DIAG ~ AGE + C(SEX) + C(CP) + C(FBS) + TRESTBPS + CHOL +'
                         'C(RESTECG) + THALACH + C(EXANG) + OLDPEAK + C(SLOPE) +'
                         'CA + C(THAL) - 1', df, return_type='dataframe')

    # The y (target) matrix uses dummy encoding for both True and False values, 
    #  since we only need to know if TRUE or not we can drop it
    y.drop(columns=['DIAG[False]'],inplace=True)
    y.rename(columns={'DIAG[True':'DIAG'}, inplace=True)

    # For convenience we'll rename all the other columns so that we can read them
    X.rename(columns={'C(SEX)[T.1]':'Male','C(CP)[T.2]':'CP_Atypical',
                      'C(CP)[T.3]':'CP_NonAngina', 'C(CP)[T.4]':'CP_Asymptomatic',
                      'C(FBS)[T.True]':'FBS_True', 'C(RESTECG)[T.1]':'RESTECG_1',
                      'C(RESTECG)[T.2]':'RESTECG_2', 'C(EXANG)[T.1]':'EXANG_True',
                      'C(SLOPE)[T.2]':'SLOPE_Flat', 'C(SLOPE)[T.3]':'SLOPE_Down',
                      'C(THAL)[T.6.0]':'THAL_FIXED', 'C(THAL)[T.7.0]':'THAL_REV'},
            inplace=True)

    print(f'X_shape:{X.shape}, y_shape:{y.shape}')
    y = np.ravel(y)
    return X, y

# It's convenient to have the preceding in a function so that we can call it again later if needed
X, y = setup_matricies(heart_df)

X_shape:(297, 19), y_shape:(297, 1)


In [19]:
# by setting the random_state variable, we can ensure that each time we run the method 
# we get the same splits
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=123)

In [24]:
# Now let's see if we can find a good model just by varying the values of K
best_score = 0
best_k = 0
clf = None
for K in range(1, MAX_NEIGHBORS, 2):
    # We are only going to change the number of neighbors on each pass to see if it helps
    clf = KNeighborsClassifier(n_neighbors=K, weights='uniform', algorithm='auto', n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_valid)
    score = accuracy_score(y_valid, y_pred)
    if score > best_score:
        best_score = score
        best_k = K
        best_y_pred = y_pred
print(f'Best for K (simple) = {best_k} Score: {best_score:.6f}')

Best for K (simple) = 7 Score: 0.663866


## Cross-Validation
Another technique that helps us to ensure that we are not overfitting is to take several passes at the data and leave out some percentage.  For instance, if we have 5 passes, we split the data into 5 groups.  In pass one, we use groups 1-4 to train the model and group 5 to validate.  Then we train on sets 1-3 and 5 and validate against 4.  Each time leaving 1/5 of the data to validate our model.  When we are done, we take the average of each of the splits and hopefully have a better model.

We can cross-validate in a number of ways:
* Leave one out cross validation
* k-fold cross validation
* Stratified k-fold cross validation
* Time Series cross validation

In [ ]:
best_c_val = 0
best_k = 0
best_model = None
for n in range(1, MAX_NEIGHBORS, 2):
    clf = KNeighborsClassifier(n_neighbors=n)
    c_val = cross_val_score(clf, X, y, cv=5, scoring='accuracy').mean()
    if c_val > best_c_val:
        best_c_val = c_val
        best_k = n
        best_model = clf
print(f'Best for K (cross_val) = {best_k} Score: {best_c_val:.6f}')
return best_model